# 0.0 Imports

In [1]:
import pandas            as pd
import numpy             as np
import seaborn           as sns
import re
import umap.umap_ as umap
from sklearn import mixture as mx
from scipy.cluster import hierarchy as hc
from matplotlib          import pyplot as plt
from sklearn             import cluster as c
from sklearn import metrics as m
from sklearn.manifold import TSNE
from plotly import express as px
from sklearn import ensemble 
from sklearn import preprocessing as pp

## 0.2 Load Data

In [2]:
df_raw = pd.read_csv('../data/Ecommerce.csv')

In [3]:
# droping extra column
df_raw.drop(columns=['Unnamed: 8'], axis=1, inplace=True)

# 1.0 Data Description

In [4]:
df1 = df_raw.copy()

## 1.1 Rename columns

In [5]:
df1.columns = ['invoice_no','stock_code','description','quantity','invoice_date','unit_price','customer_id','country']

## 1.2 Data Dimensions

In [6]:
df1.shape

(541909, 8)

## 1.3 Data types

In [7]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

## 1.4 Check Na

In [8]:
df1.isnull().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

## 1.5 Replace Na

In [9]:
df_missing = df1.loc[df1['customer_id'].isna(), :]
df_not_missing = df1.loc[~df1['customer_id'].isna(), :]

In [10]:
# create reference 
df_backup = pd.DataFrame(df_missing['invoice_no'].drop_duplicates())
df_backup['customer_id']  = np.arange(19000, 19000+len(df_backup),1)

# merge original with reference dataframe
df1 = pd.merge(df1, df_backup, on='invoice_no', how='left')

# coalesce
df1['customer_id'] = df1['customer_id_x'].combine_first(df1['customer_id_y'])

# drop extra columns
df1 = df1.drop(columns=['customer_id_x','customer_id_y'], axis=1)

In [11]:
df1.isna().sum()

invoice_no         0
stock_code         0
description     1454
quantity           0
invoice_date       0
unit_price         0
country            0
customer_id        0
dtype: int64

## 1.6 Change Types

In [12]:
# invoice date
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'], format='%d-%b-%y')

# customer_id
df1['customer_id'] = df1['customer_id'].astype('int64')

In [13]:
df1.dtypes

invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
country                 object
customer_id              int64
dtype: object

## 1.7 Descriptive Statistics 

In [14]:
num_attributes = df1.select_dtypes(include=['int64','float64'])
cat_attributes = df1.select_dtypes(include='object')

### 1.7.1 Numerical Atributtes

In [15]:
# central tendency - mean, median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

# dispersion - min, max, range, std, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.min)).T
d2 = pd.DataFrame(num_attributes.apply(np.max)).T
d3 = pd.DataFrame(num_attributes.apply(np.std)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

a = pd.concat([d1,d2,d4,d3,ct1,ct2,d5,d6]).T.reset_index()

a.columns = ['attributes','min','max','range','std','mean','median','skew','kurtosis']
a

,attributes,min,max,range,std,mean,median,skew,kurtosis
0,quantity,-80995.00,80995.0,161990.00,218.080957,9.552250,3.00,-0.264076,119769.160031
1,unit_price,-11062.06,38970.0,50032.06,96.759764,4.611114,2.08,186.506972,59005.719097
2,customer_id,12346.00,22709.0,10363.00,2911.408666,16688.840453,16249.00,0.487449,-0.804287


### 1.7.2 Categorical Atributtes

In [16]:
# invoice number
len(cat_attributes.loc[cat_attributes['invoice_no'].apply(lambda x: bool(re.search('[^0-9]+', x))), 'invoice_no'].drop_duplicates())

3839

In [17]:
# stock code
cat_attributes.loc[cat_attributes['stock_code'].apply(lambda x: bool(re.search('^[a-zA-Z]+$',x))), 'stock_code'].unique()

# Ação: 1. Remove stock_code in ['POST', 'D', 'M', 'PADS', 'DOT', 'CRUK']

array(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY',
       'DCGSSGIRL', 'PADS', 'B', 'CRUK'], dtype=object)

# 2.0 Variable Filtering

In [18]:
df2 = df1.copy()

In [19]:
# unit price > 0.0
df2 = df2.loc[df2['unit_price'] > 0.04, :]

# stock code != [POST, D, M, DOT, CRUK]
df2 = df2[~df2['stock_code'].isin(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK'])]

# description
df2 = df2.drop(columns='description', axis=1)

# map
df2 = df2[~df2['country'].isin(['European Community','Unspecified'])]

# bad users
df2 = df2[~df2['customer_id'].isin([16446])]

# quantity - negative numbers means product returns
df2_returns = df2.loc[df2['quantity'] < 0, :]
df2_purchase = df2.loc[df2['quantity'] >0, :]

# 3.0 Feature Engineering

In [20]:
df3 = df2.copy()

## 3.1 Feature Creation

In [21]:
df_ref = df3.drop(['invoice_no','stock_code','quantity',
                   'invoice_date','unit_price','country'], axis=1).drop_duplicates(ignore_index=True)

### 3.1.1 Gross Revenue

In [22]:
# Gross Revenue (Faturamento)
df2_purchase.loc[:, 'gross_revenue'] =  df2_purchase.loc[:, 'quantity'] * df2_purchase.loc[:, 'unit_price']

# Monetary
df_monetary = df2_purchase.loc[:, ['customer_id','gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge(df_ref, df_monetary, on='customer_id', how='left')
df_ref.isna().sum()

C:\Users\leona\anaconda3\envs\insiders_clustering\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\Users\leona\anaconda3\envs\insiders_clustering\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


customer_id       0
gross_revenue    91
dtype: int64

### 3.1.2 Recency - Day from last purchase

In [23]:
# Recency - last day purchase
df_recency = df2_purchase.loc[:, ['customer_id','invoice_date']].groupby('customer_id').max().reset_index()
df_recency['recency_days'] = (df2['invoice_date'].max() - df_recency['invoice_date']).dt.days
df_recency = df_recency[['customer_id','recency_days']].copy()
df_ref = pd.merge(df_ref, df_recency, on='customer_id', how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
dtype: int64

### 3.1.5 Quantity of products purchased

In [24]:
df_freq = (df2_purchase.loc[:, ['customer_id','stock_code']].groupby('customer_id').count().reset_index().rename(columns={'stock_code':'qtde_products'}))
df_ref = pd.merge(df_ref, df_freq, on='customer_id', how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qtde_products    91
dtype: int64

### 3.1.8 Frequency Purchase

In [25]:
df_aux = (df2_purchase[['customer_id','invoice_no','invoice_date']].drop_duplicates().groupby('customer_id').agg(max_ = ('invoice_date','max'),
                                                                                                                 min_ = ('invoice_date','min'),
                                                                                                                 days_ = ('invoice_date', lambda x: ((x.max()-x.min()).days) +1),
                                                                                                                 buy_ = ('invoice_date','count'))).reset_index()
# frequency
df_aux['frequency'] = df_aux[['buy_','days_']].apply(lambda x: x['buy_']/x['days_'] if x['days_'] != 0 else 0, axis=1)

# merge
df_ref = pd.merge(df_ref, df_aux[['customer_id','frequency']], on='customer_id', how='left')

df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qtde_products    91
frequency        91
dtype: int64

### 3.1.9 Number of returns

In [26]:
df_returns = df2_returns[['customer_id','quantity']].groupby('customer_id').sum().reset_index().rename(columns={'quantity':'qtde_returns'})
df_returns['qtde_returns'] = df_returns['qtde_returns'] * -1

df_ref = pd.merge(df_ref, df_returns, how='left', on='customer_id')
df_ref.loc[df_ref['qtde_returns'].isna(), 'qtde_returns'] = 0

df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qtde_products    91
frequency        91
qtde_returns      0
dtype: int64

# 4.0 EDA

In [27]:
df4 = df_ref.dropna()

## 4.3 Estudo do Espaço

In [28]:
# select dataset
cols_selected = ['customer_id','gross_revenue','recency_days','qtde_products','frequency','qtde_returns']
df43 = df4[cols_selected].drop(columns='customer_id', axis=1)

In [29]:
mm = pp.MinMaxScaler()

df43['gross_revenue'] = mm.fit_transform(df43[['gross_revenue']])
df43['recency_days'] = mm.fit_transform(df43[['recency_days']])
df43['qtde_products'] = mm.fit_transform(df43[['qtde_products']])
df43['frequency'] = mm.fit_transform(df43[['frequency']].values)
df43['qtde_returns'] = mm.fit_transform(df43[['qtde_returns']])

X = df43.copy()

### 4.3.4 Tree based embedding

In [30]:
X = df43.drop(columns=['gross_revenue'], axis=1)
y = df43['gross_revenue']

In [31]:
X = df43.drop(columns=['gross_revenue'], axis=1)
y = df43['gross_revenue']

# model definition
rf_model = ensemble.RandomForestRegressor(n_estimators=882, random_state=42, max_depth=138, min_samples_split=4, 
                                          min_samples_leaf = 3, max_features='sqrt')

# model training
rf_model.fit(X,y)

# dataframe leaf
df_leaf = pd.DataFrame(rf_model.apply(X))

In [32]:
# reduzir dimensionalidade

reducer = umap.UMAP(random_state=42)
embedding = reducer.fit_transform(df_leaf)

# embedding
df_tree = pd.DataFrame()
df_tree['embedding_x'] = embedding[:, 0]
df_tree['embedding_y'] = embedding[:, 1]

# 7.0 Hyperparameter Fine_Tunning

In [33]:
X = df_tree.copy()

# 8.0 Model Training

## 8.1 Final Model

### K-Means

In [34]:
# model definition
k = 9
kmeans = c.KMeans(init='random', n_clusters=k, n_init=10, max_iter=300, random_state=42)

# model traiing
kmeans.fit(X)

# clustering
labels =kmeans.labels_

## 8.2 Cluster Validation

In [35]:
## SS (Silhouette Score)
print('SS value: {}'.format(m.silhouette_score(X, labels, metric='euclidean')))

SS value: 0.5391868352890015


# 9.0 Cluster Analysis

## 9.4 Cluster Profile

In [36]:
df92 = df4[cols_selected].copy()
df92['cluster'] = labels
df92.head()

# change data types
df92['recency_days'] = df92['recency_days'].astype(int)
df92['qtde_products'] = df92['qtde_products'].astype(int)
df92['qtde_returns'] = df92['qtde_returns'].astype(int)

In [37]:
# Number of customers
df_cluster = df92[['customer_id','cluster']].groupby('cluster').count().reset_index()
df_cluster['perc_customer'] = 100*(df_cluster['customer_id'] / df_cluster['customer_id'].sum())

# Avg gross revenue
df_avg_gross_revenue = df92[['gross_revenue','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_gross_revenue, how='inner', on='cluster')

# Avg recency days
df_avg_recency_days = df92[['recency_days','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_recency_days, how='inner', on='cluster')

# Avg invoice_no
df_qtde_products = df92[['qtde_products','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_qtde_products, how='inner', on='cluster')

# frequency 
df_frequency = df92[['frequency','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_frequency, how='inner', on='cluster')

# returns
df_qtde_returns = df92[['qtde_returns','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_qtde_returns, how='inner', on='cluster')

df_cluster.sort_values('gross_revenue', ascending=False)

,cluster,customer_id,perc_customer,gross_revenue,recency_days,qtde_products,frequency,qtde_returns
3,3,735,12.906058,5621.197932,8.103401,232.839456,0.437987,51.038095
8,8,664,11.659350,3227.552771,46.611446,214.234940,0.061802,20.003012
0,0,345,6.057946,3048.694232,218.081159,236.600000,1.028069,307.797101
5,5,889,15.610184,1344.247390,49.436445,57.813273,0.030252,18.434196
6,6,371,6.514486,938.380782,106.889488,42.423181,0.026668,0.339623
4,4,780,13.696225,861.599269,173.894872,57.461538,1.009909,0.443590
1,1,345,6.057946,592.842203,88.521739,14.376812,0.029635,0.263768
2,2,933,16.382792,237.770975,262.662379,10.672026,1.008322,4.375134
7,7,633,11.115013,215.238246,93.072670,8.620853,0.985292,0.086888


# 11.0 Deploy

## 11.1 Insert into SQLITE

In [38]:
import sqlite3
from sqlalchemy import create_engine

In [39]:
# create table
query_create_table_insiders = """
    CREATE TABLE insiders (
    customer_id      INTEGER,
    gross_revenue    REAL,
    recency_days     INTEGER,
    qtde_products    INTEGER,
    frequency        REAL,
    qtde_returns     INTEGER,
    cluster          INTEGER  
    )
"""
conn = sqlite3.connect('insiders_db.sqlite')
conn.execute(query_create_table_insiders)
conn.commit()
#conn.close()

# insert data
conn = create_engine('sqlite:///insiders_db.sqlite')
df92.to_sql('insiders', con=conn, if_exists='append', index=False)

In [44]:
pwd

'C:\\Users\\leona\\Documents\\Data Science\\Portfolio projects\\insiders-clustering\\notebooks'